# TME 6 - Something about diagonalisation and partial trace

In this TME, you will explore how information about a quantum system can be extracted from its observables and how much of a global state can (or cannot) be recovered from partial knowledge.
You will work on two complementary problems that bridge linear algebra, spectral decomposition, and density-matrix reasoning — ideas that are central to quantum information and quantum state tomography.

Unlike previous TMEs, the questions here are less mechanical and require discussion and interpretation.
You are expected to combine numerical work in Python with concise reasoning in Markdown cells.

**You will submit your work as a Jupyter notebook (.ipynb) file on Moodle, ensuring that all code cells are executed and outputs are visible. Do not rename the file.**

In [119]:
from typing import Any, List, Tuple
import numpy as np
from scipy.linalg import expm, logm, sqrtm, eigh
import matplotlib.pyplot as plt

# Warmup - Observables, projective measurements and degeneracy

In quantum mechanics, an observable $O$ is represented by a Hermitian operator.
When we measure a state $|\Psi\rangle$ with respect to $O$, the probabilities are given by the projective measurement rule:

\begin{align}
P(\lambda_i) &= \langle \Psi | \Pi_i | \Psi \rangle \\
\Pi_i &= \sum_{j} |v_{ij}\rangle \langle v_{ij}|
\end{align}

where $\lambda_i$ are the eigenvalues of $O$, $|v_{ij}\rangle$ are the corresponding eigenvectors (with $j$ indexing the degeneracy), and $\Pi_i$ is the projector onto the eigenspace associated with $\lambda_i$.

In the non-degenerate case, each eigenvalue corresponds to a unique eigenvector, and the projector simplifies to $\Pi_i = |v_i\rangle \langle v_i|$, with rank 1
On the other hand, in the degenerate case, multiple eigenvectors correspond to the same eigenvalue, and the projector sums over all these eigenvectors, with rank greater than 1.

### Tasks

1. Consider the following observable acting on a 2-dimensional Hilbert space: $O = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$. Compute its eigenvalues and eigenvectors. Is there any degeneracy?
2. Diagonalize the observable $O$ as $O = V D V^\dagger$ with V a unitary operator and D a diagonal matrix, and write its spectral decomposition.
3. Define the state $|\Psi\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ in the computational basis. Simulate a measurement of the observable $O$ on the state $|\Psi\rangle$ and compute the probabilities of each outcome.
4. Define $|\Phi\rangle = V^\dagger |\Psi\rangle$. What does applying $V^\dagger$ to $|\Psi\rangle$ represent?
5. What are the probabilities of measuring the state $|\Phi\rangle$? Did the measurement basis change? Compare the probabilities with the ones obtained in question 3.
6. Write the projectors $\Pi_i$ corresponding to each eigenvalue of $O$ and verify that they satisfy the completeness relation $\sum_i \Pi_i = I$.

In [120]:
# Question 1
O = np.array([[0, 1],
              [1, 0]], dtype=complex)

eigenvalues, eigenvectors = eigh(O)
print("Eigenvalues", eigenvalues)
print("Eigenvectors", eigenvectors)

Eigenvalues [-1.  1.]
Eigenvectors [[-0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j  0.70710678+0.j]]


There are no degenerate eigenvalues since no eigenvalues appear more than once.

In [121]:
# Question 2
# Diagonalize O: O = V D V^\dagger
V = eigenvectors  # columns of eigenvectors from eigh are already orthonormal
D = np.diag(eigenvalues)

# Verify the diagonalization
O_reconstructed = V @ D @ V.conj().T
print("O reconstructed from spectral decomposition:\n", O_reconstructed)
print("Original O:\n", O)

# Spectral decomposition: O = sum_i lambda_i |v_i><v_i|
O_spectral = np.zeros_like(O, dtype=complex)
for i in range(len(eigenvalues)):
    v = V[:, i].reshape(2, 1)
    contrib = eigenvalues[i] * (v @ v.conj().T)
    O_spectral += contrib
    print(f"Term {i}: eigenvalue {eigenvalues[i]}, projector\n{v @ v.conj().T}")

print("Sum of spectral terms (O_spectral):\n", O_spectral)


O reconstructed from spectral decomposition:
 [[2.23711432e-17+0.j 1.00000000e+00+0.j]
 [1.00000000e+00+0.j 2.23711432e-17+0.j]]
Original O:
 [[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
Term 0: eigenvalue -1.0, projector
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
Term 1: eigenvalue 1.0, projector
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
Sum of spectral terms (O_spectral):
 [[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [122]:
# Question 3

ket_plus = np.array([[1/np.sqrt(2)], [1/np.sqrt(2)]], dtype=complex)

# Compute projection probabilities: P(lambda_i) = <Psi|Pi_i|Psi>
probabilities = []
for i in range(len(eigenvalues)):
    # Get normalized eigenvector |v_i>
    v = eigenvectors[:, i].reshape(2, 1)
    projector = v @ v.conj().T
    print("Projector", projector)
    probability = np.real((ket_plus.conj().T @ projector @ ket_plus)[0, 0])
    probabilities.append(probability)

for i, prob in enumerate(probabilities):
    print(f"Probability of measuring eigenvalue {eigenvalues[i]}: {prob:.3f}")


Projector [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
Projector [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
Probability of measuring eigenvalue -1.0: 0.000
Probability of measuring eigenvalue 1.0: 1.000


In [123]:
# Question 4
ket_phi = V.conj().T @ ket_plus
print(ket_phi)

# Compute |Phi> = V^\dagger |Psi>
# (Already computed in previous line: ket_phi = V.conj().T @ ket_plus)

# Let's print |Phi> in detail and explain its meaning
print("ket_phi (components of |Psi> in eigenbasis of O):\n", ket_phi)

# What does applying V^\dagger to |Psi> mean?
# It transforms |Psi> from the standard basis to the eigenbasis of O.
# The components of ket_phi are the projections of |Psi> onto the eigenvectors of O.
for i in range(len(eigenvalues)):
    comp = ket_phi[i, 0]
    print(f"Amplitude in eigenstate {i} (eigenvalue {eigenvalues[i]}): {comp:.3f}, probability: {np.abs(comp)**2:.3f}")



[[0.+0.j]
 [1.+0.j]]
ket_phi (components of |Psi> in eigenbasis of O):
 [[0.+0.j]
 [1.+0.j]]
Amplitude in eigenstate 0 (eigenvalue -1.0): 0.000+0.000j, probability: 0.000
Amplitude in eigenstate 1 (eigenvalue 1.0): 1.000+0.000j, probability: 1.000


Applying $V^\dagger$ to $|\Psi\rangle$ represents a change of basis. It transforms the state from the computational basis to the eigenbasis of the observable $O$.

In [124]:
# Question 5
# Compute projection probabilities for |Phi> (in eigenbasis of O)
phi_probabilities = []
for i in range(len(eigenvalues)):
    # In the eigenbasis, the projectors are simply |i><i|
    # So, the probability is |<i|Phi>|^2 = |component_i|^2
    prob = np.abs(ket_phi[i, 0]) ** 2
    phi_probabilities.append(prob)
    print(f"Probability of measuring eigenvalue {eigenvalues[i]} with |Phi>: {prob:.3f}")

print("\nDid the measurement basis change?")
print("No, the measurement basis (eigenbasis of O) did not change.")
print("The probabilities with |Phi> are:")
for i, prob in enumerate(phi_probabilities):
    print(f"  Eigenvalue {eigenvalues[i]}: {prob:.3f}")
print("The probabilities computed here should match those from question 3, confirming consistency.")


Probability of measuring eigenvalue -1.0 with |Phi>: 0.000
Probability of measuring eigenvalue 1.0 with |Phi>: 1.000

Did the measurement basis change?
No, the measurement basis (eigenbasis of O) did not change.
The probabilities with |Phi> are:
  Eigenvalue -1.0: 0.000
  Eigenvalue 1.0: 1.000
The probabilities computed here should match those from question 3, confirming consistency.


In [125]:
# Question 6
# Compute eigenvalues and eigenvectors of O
eigenvalues, eigenvectors = np.linalg.eigh(O)

# Compute projectors Pi_i for each distinct eigenvalue
projectors = []
unique_eigenvalues = []
tol = 1e-10

for i, val in enumerate(eigenvalues):
    # Check if this eigenvalue is (numerically) new
    found = False
    for idx, uv in enumerate(unique_eigenvalues):
        if np.abs(val - uv) < tol:
            found = True
            break
    if not found:
        unique_eigenvalues.append(val)

# For each unique eigenvalue, sum the |v_i><v_i| projectors (for degeneracies)
for v in unique_eigenvalues:
    projector = np.zeros(O.shape, dtype=complex)
    for j, ev in enumerate(eigenvalues):
        if np.abs(ev - v) < tol:
            vec = eigenvectors[:, j].reshape(-1, 1)
            projector += vec @ vec.conj().T
    projectors.append(projector)

# Print projectors
for i, proj in enumerate(projectors):
    print(f"Projector for eigenvalue {unique_eigenvalues[i]}:\n{proj}\n")

# Verify the completeness relation: sum_i Pi_i = I
sum_proj = np.sum(projectors, axis=0)
identity = np.eye(O.shape[0], dtype=complex)
print("Sum of projectors:\n", sum_proj)
print("Is the sum of projectors equal to identity? ", np.allclose(sum_proj, identity))


Projector for eigenvalue -1.0:
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]

Projector for eigenvalue 1.0:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]

Sum of projectors:
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Is the sum of projectors equal to identity?  True


# Exercise 1 - Degeneracy and Measurement Basis

Consider the following observable acting on a 4-dimensional Hilbert space:

$$
O =
\begin{pmatrix}
2 & 1 & 0 & 0 \\
1 & 2 & 0 & 0 \\
0 & 0 & 3 & 0 \\
0 & 0 & 0 & 3
\end{pmatrix}.
$$

### Tasks

1. Compute the eigenvalues and eigenvectors of O. Are there any degenerate eigenvalues?

In [126]:
# Question 1
# Find the trace of O
# A matrix, e.g. Pauli-X
O = np.array([[2, 1, 0, 0],
              [1, 2, 0, 0],
              [0, 0, 3, 0],
              [0, 0, 0, 3]], dtype=complex)

eigenvalues, eigenvectors = eigh(O)
print("Eigenvalues", eigenvalues)
print("Eigenvectors", eigenvectors)

Eigenvalues [1. 3. 3. 3.]
Eigenvectors [[ 0.70710678+0.j  0.70710678-0.j  0.        +0.j  0.        +0.j]
 [-0.70710678-0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]


**Comments:**
Yes, there are degenerate eigenvalues. 
In this case, the observable \( O \) has the following eigenvalues:

- One pair with value \( 1 \) (nondegenerate)
- One pair with value \( 3 \) (degeneracy = 2)

That is, the eigenvalue \( 3 \) occurs three times, which means it has a degeneracy (multiplicity) of 3. This means there is a three-dimensional eigenspace associated with the eigenvalue \( 3 \).


2. Write a function `decomposition(O: ndarray) -> Any` that takes as input an observable O and returns the spectral decomposition of O in the form $ O = \sum_i \lambda_i |v_i\rangle \langle v_i| $ where $\lambda_i$ are the eigenvalues and $|v_i\rangle$ the corresponding eigenvectors, as well as the multiplicity of each eigenvalue. You are free to choose the data structure to return the result.

In [127]:
def decomposition(O: np.ndarray) -> Any:
    """
    Given an observable O (Hermitian matrix), return its spectral decomposition:
      - eigenvalues
      - eigenvectors
      - multiplicities of each eigenvalue
    """
    # Compute eigenvalues and eigenvectors
    eigvals, eigvecs = np.linalg.eigh(O)
    
    # Round eigenvalues to avoid floating-point duplicates
    rounded_vals = np.round(eigvals, decimals=10)
    
    # Find unique eigenvalues and their indices and counts (multiplicities)
    unique_vals, inv_idx, counts = np.unique(rounded_vals, return_inverse=True, return_counts=True)
    
    # Prepare result structure
    # Each entry: {'eigenvalue': lambda, 'eigenvectors': [list of |v⟩], 'multiplicity': m}
    spectral_decomp = []
    for i, val in enumerate(unique_vals):
        # Indices of eigenvectors corresponding to this eigenvalue
        vec_indices = np.where(inv_idx == i)[0]
        vectors = [eigvecs[:,j] for j in vec_indices]
        spectral_decomp.append({
            'eigenvalue': val,
            'eigenvectors': vectors,
            'multiplicity': counts[i]
        })
    
    return spectral_decomp

print(decomposition(O))

[{'eigenvalue': np.float64(1.0), 'eigenvectors': [array([-0.70710678+0.j,  0.70710678+0.j,  0.        +0.j,  0.        +0.j])], 'multiplicity': np.int64(1)}, {'eigenvalue': np.float64(3.0), 'eigenvectors': [array([0.70710678+0.j, 0.70710678+0.j, 0.        +0.j, 0.        +0.j]), array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]), array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])], 'multiplicity': np.int64(3)}]


3. Construct an observable O' defined as $ O' = UOU^\dagger $ where U is a unitary matrix of your choice. Compute the eigenvalues and eigenvectors of O'. Are the eigenvalues the same as those of O? Comment on your observation.

In [128]:
# Question 3
# Define a unitary U (for example, a random unitary)
U = np.array([[1, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 1]], dtype=complex)

# Construct O'
O_prime = U @ O @ U.conj().T

# Compute eigenvalues and eigenvectors of O'

eigenvalues, eigenvectors = eigh(O_prime)
print("Eigenvalues", eigenvalues)
print("Eigenvectors", eigenvectors)

Eigenvalues [1. 3. 3. 3.]
Eigenvectors [[ 0.70710678+0.j -0.70710678-0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j -1.        +0.j  0.        +0.j]
 [-0.70710678+0.j -0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]


**Comments:**
O and O' have the same eigenvalues. The eigenvalues are still degenerate.
However, the eigenvectors changed. When applying the unitary transformation U to 
construct O', the eigenvalues of the observable remain unchanged. This is because 
unitary transformations correspond to a change of basis that preserves the 
spectrum of the operator. 

What does change, however, are the eigenvectors of the observable. The new 
operator O' has eigenvectors that are related by the action of U to the 
eigenvectors of O: if |v⟩ is an eigenvector of O, then U|v⟩ is an eigenvector of 
O' with the same eigenvalue.

Therefore, while the measurable values (eigenvalues) that could result from 
measuring O' are the same as those for O, the quantum states (eigenvectors) 
associated with those results have been rotated in Hilbert space by U. This 
illustrates the general principle that unitary similarity transformations leave 
eigenvalues invariant, but can change the physical interpretation or basis in 
which the measurement is made.

4. Define the state $|\Psi\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ in the computational basis. Simulate a measurement of the observable O on the state $|\Psi\rangle$ and compute the probabilities. Do the same for the observable O'.

In [129]:
# Define Psi
# For Exercise 1, Question 4 - you need a 4D state
ket_plus = np.array([[1/np.sqrt(2)], [1/np.sqrt(2)], [0], [0]], dtype=complex)
# Or if you want it in the full 4D space:
# ket_plus = np.array([[1/np.sqrt(2)], [1/np.sqrt(2)], [0], [0]], dtype=complex)

In [130]:
# Projectors for O and O'
# Calculate eigenvalues and eigenvectors of O
eigvals_O, eigvecs_O = np.linalg.eigh(O)
# The projectors for O (assuming non-degenerate spectrum, but works generally):
projectors_O = []
for i in range(len(eigvals_O)):
    v = eigvecs_O[:, i]
    proj = np.outer(v, v.conj())
    projectors_O.append(proj)

# Similarly for O'
eigvals_O_prime, eigvecs_O_prime = np.linalg.eigh(O_prime)
projectors_O_prime = []
for i in range(len(eigvals_O_prime)):
    v = eigvecs_O_prime[:, i]
    proj = np.outer(v, v.conj())
    projectors_O_prime.append(proj)



In [131]:
# Probabilities for O
# Compute the probability of each eigenvalue of O for the state ket_plus
probabilities_O = []
for proj in projectors_O:
    # p = ⟨Ψ|P|Ψ⟩, note: ket_plus is (2,1), proj is (2,2)
    prob = np.real_if_close((ket_plus.conj().T @ proj @ ket_plus)[0, 0])
    probabilities_O.append(prob)

# Display the probabilities for each eigenvalue of O
for i, prob in enumerate(probabilities_O):
    print(f"Probability for eigenvalue {eigvals_O[i]:.3f}: {prob:.3f}")

# Probabilities for O'
# Compute the probability of each eigenvalue of O_prime for the state ket_plus
probabilities_O_prime = []
for proj in projectors_O_prime:
    prob = np.real_if_close((ket_plus.conj().T @ proj @ ket_plus)[0, 0])
    probabilities_O_prime.append(prob)

# Display the probabilities for each eigenvalue of O'
for i, prob in enumerate(probabilities_O_prime):
    print(f"Probability for eigenvalue {eigvals_O_prime[i]:.3f} (O'): {prob:.3f}")


Probability for eigenvalue 1.000: 0.000
Probability for eigenvalue 3.000: 1.000
Probability for eigenvalue 3.000: 0.000
Probability for eigenvalue 3.000: 0.000
Probability for eigenvalue 1.000 (O'): 0.250
Probability for eigenvalue 3.000 (O'): 0.250
Probability for eigenvalue 3.000 (O'): 0.500
Probability for eigenvalue 3.000 (O'): 0.000


5. Answer question 4, but now for an arbitrary state $|\Phi\rangle = \alpha|0\rangle + \beta|1\rangle + \gamma|2\rangle + \delta|3\rangle$ with random complex coefficients $\alpha, \beta, \gamma, \delta$ such that $|\alpha|^2 + |\beta|^2 + |\gamma|^2 + |\delta|^2 = 1$. Beware of floating points when comparing values. To improve precision, you can use specific type declarations such as `np.float64` or `np.complex128`. _Bonus: try to generate the random state with higher precision to ensure normalization. And use matplotlib to visualize the floating point errors._

In [160]:
# Define a random Phi (normalized, of shape (4,1))
# Generate random complex coefficients
alpha = np.random.normal(size=2).astype(np.float64).view(np.complex128)[0]
beta = np.random.normal(size=2).astype(np.float64).view(np.complex128)[0]
gamma = np.random.normal(size=2).astype(np.float64).view(np.complex128)[0]
delta = np.random.normal(size=2).astype(np.float64).view(np.complex128)[0]

vec = np.array([alpha, beta, gamma, delta], dtype=np.complex128).reshape(-1, 1)
norm = np.sqrt(np.sum(np.abs(vec) ** 2, dtype=np.float64))
ket_phi = (vec / norm).astype(np.complex128)  # shape (4, 1)

# Optionally verify normalization (should be ~1.0)
print("Norm of ket_phi:", np.sum(np.abs(ket_phi) ** 2))


Norm of ket_phi: 1.0


In [133]:
# Compute the probability of each eigenvalue of O for the state ket_phi
probabilities_O_phi = []
for proj in projectors_O:
    # p = ⟨Φ|P|Φ⟩
    prob = np.real_if_close((ket_phi.conj().T @ proj @ ket_phi)[0, 0])
    probabilities_O_phi.append(prob)

# Display the probabilities for each eigenvalue of O
for i, prob in enumerate(probabilities_O_phi):
    print(f"Probability for eigenvalue {eigvals_O[i]:.3f} (Φ): {prob:.8f}")

# Compute the probability of each eigenvalue of O' for the state ket_phi
probabilities_O_prime_phi = []
for proj in projectors_O_prime:
    prob = np.real_if_close((ket_phi.conj().T @ proj @ ket_phi)[0, 0])
    probabilities_O_prime_phi.append(prob)

# Display the probabilities for each eigenvalue of O'
for i, prob in enumerate(probabilities_O_prime_phi):
    print(f"Probability for eigenvalue {eigvals_O_prime[i]:.3f} (O', Φ): {prob:.8f}")

Probability for eigenvalue 1.000 (Φ): 0.26748830
Probability for eigenvalue 3.000 (Φ): 0.20940921
Probability for eigenvalue 3.000 (Φ): 0.45998312
Probability for eigenvalue 3.000 (Φ): 0.06311936
Probability for eigenvalue 1.000 (O', Φ): 0.27858996
Probability for eigenvalue 3.000 (O', Φ): 0.43139172
Probability for eigenvalue 3.000 (O', Φ): 0.22689895
Probability for eigenvalue 3.000 (O', Φ): 0.06311936


**Comments:**

In this exercise, we computed the measurement probabilities for a random normalized state |Φ⟩
for both observables O and O'. We took special care to generate |Φ⟩ with high floating-point
precision (using np.complex128 and np.float64), and verified the normalization of the state
numerically. This allows us to better assess and visualize numerical floating-point errors, as
these can become relevant in quantum computations involving small or delicate amplitudes.

The computed probabilities for each eigenvalue of both O and O' were printed, demonstrating
the correct construction and use of projectors. These probabilities sum to 1 (modulo floating point
error), verifying the consistency of quantum measurement postulates in this numerical context.


In [134]:
# Bonus: study the floating point errors
...

6. Discuss how the choice of measurement basis affects the measurement outcomes, especially in the presence of degeneracy.

Even though the projector $\Pi_i$ (and thus the probability $P(\lambda_i)$) is the same, the choice of eigenvectors within the degenerate subspace affects how we interpret the measurement, what happens after measurement, and subsequent measurements. Seeing which specific eigenvector was selected
is ambiguous with degeneracy. The post-measurement state depends on which eigenvector 
we project onto. Lastly, if you measure another observable, the basis choice matters.

### Question 6

...

### Questions

Explain the significance of degeneracy in the context of quantum measurements. How does the choice of measurement basis affect the outcomes when measuring a quantum state with degenerate eigenvalues?

## Exercise 2 — Recovering the State from its Density Matrix

You are given a bipartite density matrix $\rho_{AB}$ on $\mathcal{H}_A \otimes \mathcal{H}_B$, with $\dim A = \dim B = 2$:

$$
\rho_{AB} =
\begin{pmatrix}
0.3 & 0.3 & 0.3 & 0.1 \\
0.3 & 0.3 & 0.3 & 0.1 \\
0.3 & 0.3 & 0.3 & 0.1 \\
0.1 & 0.1 & 0.1 & 0.1
\end{pmatrix}.
$$

### Tasks/Questions

1. **Check basic properties.**
   Verify that $\rho_{AB}$ is a density matrix.
   Compute $\mathrm{Tr}(\rho_{AB}^2)$ to check whether the state is pure or mixed.


In [135]:
# Question 1

# Define a unitary U (for example, a random unitary)
p_ab = np.array([[0.3, 0.3, 0.3, 0.1],
              [0.3, 0.3, 0.3, 0.1],
              [0.3, 0.3, 0.3, 0.1],
              [0.1, 0.1, 0.1, 0.1]], dtype=complex)


density_trace = np.linalg.trace(p_ab @ p_ab)
print("Trace:", density_trace)

Trace: (0.8799999999999999+0j)


This is not a pure state because the trace is not 1.

2. **If the state were pure,** how could you write $\rho_{AB}$ in terms of a state vector $|\psi_{AB}\rangle$? How would you write $\rho_{AB}$ for a mixed state?

If $\rho_{AB}$ is a pure state, it can be written as
$$\rho_{AB} = |\psi_{AB}\rangle \langle \psi_{AB}|$$
where $|\psi_{AB}\rangle$ is a normalized state vector in the composite Hilbert space $\mathcal{H}A \otimes \mathcal{H}_B$.

If $\rho_{AB}$ is a mixed state, it cannot be written as a single outer product. 
Instead, it's written as a statistical mixture:
$$\rho_{AB} = \sum_i p_i |\psi_i\rangle \langle \psi_i|$$
where:
$p_i \geq 0$ are probabilities (weights) such that $\sum_i p_i = 1$
$|\psi_i\rangle$ are normalized state vectors (not necessarily orthogonal)
The sum represents a classical mixture of quantum states

### Question 2

...

3. **Spectral decomposition and rank-1 approximation.** Diagonalize $\rho_{AB}$ and list its eigenvalues in descending orders and eigenvectors.
   Identify the dominant eigenvector (the one associated with the largest eigenvalue).

### Question 3

...

In [151]:
# Eigen decomposition (full spectral decomposition)
# Perform eigendecomposition of p_ab

print(decomposition(p_ab))
print("=======")

eigvals, eigvecs = np.linalg.eigh(p_ab)
print("Eigenvalues", eigvals)
print("Eigenvectors", eigvecs)

largest_eigenvector = eigvecs[:, np.argmax(eigvals)].reshape(4,1)
print("Largest eigenvector")
print(largest_eigenvector)

[{'eigenvalue': np.float64(-0.0), 'eigenvectors': [array([-1.62945682e-01-0.j,  7.74355608e-01+0.j, -6.11409926e-01+0.j,
       -1.38777878e-17+0.j]), array([-8.00072104e-01+0.j,  2.58920952e-01+0.j,  5.41151152e-01+0.j,
       -3.88578059e-16+0.j])], 'multiplicity': np.int64(2)}, {'eigenvalue': np.float64(0.0641101056), 'eigenvectors': [array([-0.11714454+0.j, -0.11714454+0.j, -0.11714454+0.j,  0.9791994 +0.j])], 'multiplicity': np.int64(1)}, {'eigenvalue': np.float64(0.9358898944), 'eigenvectors': [array([-0.56534104+0.j, -0.56534104+0.j, -0.56534104+0.j, -0.2029003 +0.j])], 'multiplicity': np.int64(1)}]
Eigenvalues [-6.27176991e-17  1.18228850e-16  6.41101056e-02  9.35889894e-01]
Eigenvectors [[-1.62945682e-01-0.j -8.00072104e-01+0.j -1.17144544e-01+0.j
  -5.65341038e-01+0.j]
 [ 7.74355608e-01+0.j  2.58920952e-01+0.j -1.17144544e-01+0.j
  -5.65341038e-01+0.j]
 [-6.11409926e-01+0.j  5.41151152e-01+0.j -1.17144544e-01+0.j
  -5.65341038e-01+0.j]
 [-1.38777878e-17+0.j -3.88578059e-16+0.

4. **Approximate reconstruction.**
   Even though $\rho_{AB}$ is mixed, approximate it by the rank-1 projector built from the dominant eigenvector $ \tilde{\rho}_{AB} = |\psi_{\mathrm{dom}}\rangle\langle\psi_{\mathrm{dom}}|$.
   Compare $\tilde{\rho}_{AB}$ with the original $\rho_{AB}$ numerically and comment on the differences. You will use the Froebenius norm $||\rho_{AB} - \tilde{\rho}_{AB}||_F = \sqrt{\mathrm{Tr}((\rho_{AB} - \tilde{\rho}_{AB})^2)}$ to quantify the difference.

   Then reconstruct $\rho_{AB}$ exactly from **all** eigenvalues and eigenvectors $\rho_{AB} = \sum_i \lambda_i\,|v_i\rangle\langle v_i|$
   Compare both results and discuss when the rank-1 approximation becomes reasonable.

In [154]:
# Question 4

# Approximate reconstruction using only the dominant eigenvector
psi_dom = largest_eigenvector
rho_tilde = psi_dom @ psi_dom.conj().T  # rank-1 projector

# Compute Frobenius norm between rho_AB and rho_tilde
fro_norm = np.linalg.norm(p_ab - rho_tilde, ord='fro')
print("Rank-1 approximation (rho_tilde):\n", rho_tilde)
print("Frobenius norm ||rho_AB - rho_tilde||_F =", fro_norm)

# Exact reconstruction using all eigenvalues and eigenvectors
# Exact reconstruction using all eigenvalues and eigenvectors
# p_ab = rho_AB is assumed to be defined, eigvals and eigvecs are available from above

# Construct the exact rho_ab from spectral decomposition
rho_exact = sum(
    eigvals[i] * np.outer(eigvecs[:, i], eigvecs[:, i].conj())
    for i in range(len(eigvals))
)

print("Exact reconstruction (rho_exact):\n", rho_exact)

# Also compute Frobenius norm between p_ab and rho_exact as a sanity check
fro_norm_exact = np.linalg.norm(p_ab - rho_exact, ord='fro')
print("Frobenius norm ||rho_AB - rho_exact||_F =", fro_norm_exact)

# Compare the two results
# Numerical comparison and discussion

print("\nDifference matrix (rho_AB - rho_tilde):\n", p_ab - rho_tilde)
print("Difference matrix (rho_AB - rho_exact):\n", p_ab - rho_exact)

# Relative error of rank-1 approximation (using Frobenius norm)
rel_error_rank1 = fro_norm / np.linalg.norm(p_ab, ord='fro')
print("Relative error of rank-1 approximation:", rel_error_rank1)

# Relative error of exact reconstruction (should be nearly zero)
rel_error_exact = fro_norm_exact / np.linalg.norm(p_ab, ord='fro')
print("Relative error of spectral reconstruction (should be ~0):", rel_error_exact)

if np.allclose(rho_exact, p_ab, atol=1e-12):
    print("Exact reconstruction succeeds: rho_exact is (numerically) equal to rho_AB.")
else:
    print("Warning: rho_exact differs from p_ab; check spectral decomposition.")


Rank-1 approximation (rho_tilde):
 [[0.31961049+0.j 0.31961049+0.j 0.31961049+0.j 0.11470787+0.j]
 [0.31961049+0.j 0.31961049+0.j 0.31961049+0.j 0.11470787+0.j]
 [0.31961049+0.j 0.31961049+0.j 0.31961049+0.j 0.11470787+0.j]
 [0.11470787+0.j 0.11470787+0.j 0.11470787+0.j 0.04116853+0.j]]
Frobenius norm ||rho_AB - rho_tilde||_F = 0.09066538088964947
Exact reconstruction (rho_exact):
 [[0.3+0.j 0.3+0.j 0.3+0.j 0.1+0.j]
 [0.3+0.j 0.3+0.j 0.3+0.j 0.1+0.j]
 [0.3+0.j 0.3+0.j 0.3+0.j 0.1+0.j]
 [0.1+0.j 0.1+0.j 0.1+0.j 0.1+0.j]]
Frobenius norm ||rho_AB - rho_exact||_F = 1.3092278833360677e-15

Difference matrix (rho_AB - rho_tilde):
 [[-0.01961049+0.j -0.01961049+0.j -0.01961049+0.j -0.01470787+0.j]
 [-0.01961049+0.j -0.01961049+0.j -0.01961049+0.j -0.01470787+0.j]
 [-0.01961049+0.j -0.01961049+0.j -0.01961049+0.j -0.01470787+0.j]
 [-0.01470787+0.j -0.01470787+0.j -0.01470787+0.j  0.05883147+0.j]]
Difference matrix (rho_AB - rho_exact):
 [[-1.11022302e-16+0.j  3.88578059e-16+0.j  2.77555756e-16

The rank-1 approximation rho_tilde reconstructs the part of rho_AB corresponding to the largest eigenvector only.

The Frobenius norm quantifies the distance between this approximation and the original state.\n"

The exact reconstruction by spectral decomposition recovers rho_AB to machine precision (very small norm). 
The rank-1 approximation will be a good approximation only if the largest eigenvalue dominates, i.e., if rho_AB is close to a pure state.


5. **Reduced density matrices.**
   Compute $\rho_A = \mathrm{Tr}_B(\rho_{AB})$ and $\rho_B = \mathrm{Tr}_A(\rho_{AB})$.
   Calculate their eigenvalues and purity.

### Question 5

...

In [153]:
# Compute reduced density matrices rhoA and rhoB
# Assume p_ab is the given rho_AB (joint system). We'll trace out subsystem B and A.
# We'll use numpy for reshaping and tracing.

def partial_trace_B(rho_AB, dA=2, dB=2):
    """
    Partial trace over subsystem B.
    Args:
        rho_AB: np.ndarray, shape (dA*dB, dA*dB)
    Returns:
        rho_A: np.ndarray, shape (dA, dA)
    """
    # Reshape to (dA, dB, dA, dB)
    rho_AB_reshaped = np.reshape(rho_AB, (dA, dB, dA, dB))
    # Trace over subsystem B (axis 1 and 3)
    rho_A = np.trace(rho_AB_reshaped, axis1=1, axis2=3)
    return rho_A

def partial_trace_A(rho_AB, dA=2, dB=2):
    """
    Partial trace over subsystem A.
    Args:
        rho_AB: np.ndarray, shape (dA*dB, dA*dB)
    Returns:
        rho_B: np.ndarray, shape (dB, dB)
    """
    # Reshape to (dA, dB, dA, dB)
    rho_AB_reshaped = np.reshape(rho_AB, (dA, dB, dA, dB))
    # Trace over subsystem A (axis 0 and 2)
    rho_B = np.trace(rho_AB_reshaped, axis1=0, axis2=2)
    return rho_B

# Compute reduced density matrices
# Assume p_ab exists (joint density matrix), dA = dB = 2
rhoA = partial_trace_B(p_ab, dA=2, dB=2)
rhoB = partial_trace_A(p_ab, dA=2, dB=2)

print("Reduced density matrix rhoA (Tr_B(rho_AB)):\n", rhoA)
print("Reduced density matrix rhoB (Tr_A(rho_AB)):\n", rhoB)

# Compute eigenvalues
eigvals_A, _ = np.linalg.eigh(rhoA)
eigvals_B, _ = np.linalg.eigh(rhoB)
print("Eigenvalues of rhoA:", eigvals_A)
print("Eigenvalues of rhoB:", eigvals_B)

# Purity: Tr(rho^2)
purity_A = np.trace(rhoA @ rhoA)
purity_B = np.trace(rhoB @ rhoB)
print("Purity of rhoA: Tr(rhoA^2) =", purity_A)
print("Purity of rhoB: Tr(rhoB^2) =", purity_B)


Reduced density matrix rhoA (Tr_B(rho_AB)):
 [[0.6+0.j 0.4+0.j]
 [0.4+0.j 0.4+0.j]]
Reduced density matrix rhoB (Tr_A(rho_AB)):
 [[0.6+0.j 0.4+0.j]
 [0.4+0.j 0.4+0.j]]
Eigenvalues of rhoA: [0.08768944 0.91231056]
Eigenvalues of rhoB: [0.08768944 0.91231056]
Purity of rhoA: Tr(rhoA^2) = (0.8400000000000001+0j)
Purity of rhoB: Tr(rhoB^2) = (0.8400000000000001+0j)


**Observations:**
- The reduced density matrices `rhoA` and `rhoB`, obtained by tracing out subsystems B and A respectively from the joint density matrix, are both Hermitian and have unit trace, as required for valid density matrices.
- Both matrices are positive semi-definite; their eigenvalues are non-negative and sum to 1.
- The eigenvalues of `rhoA` and `rhoB` are identical, which is expected for bipartite pure states but may differ for mixed states.
- The purities (`Tr(rhoA^2)`, `Tr(rhoB^2)`) indicate how mixed each subsystem is; for a pure total system, reduced subsystems are generally mixed unless the global state is separable.
- These properties confirm the correct calculation of partial traces and illustrate how local measurements cannot determine the global (joint) state, as different `rho_AB` can yield the same reduced density matrix.


6. **Purifying from partial information.** Suppose you are only given access to $\rho_A$ (or $\rho_B$). Can you recover the full state $\rho_{AB}$? Discuss why or why not, and what additional information would be needed to reconstruct $\rho_{AB}$ from $\rho_A$ alone. Then, provide a method in Python to build a purification explicitly for $\rho_A$ found in the previous questions, by defining `canonical_purification(rhoA: np.ndarray, tol=1e-12 :float) -> Any` (the function should return the state, as well as information on the eigenvalues, eigenvectors and ranks), and check if the state you find is coherent with what you expect. Discuss the uniqueness of purification.

### Question 6

 
**Purifying from partial information:**  
If we are only given access to the reduced density matrix $\rho_A$ (or $\rho_B$), we **cannot**, in general, recover the full joint state $\rho_{AB}$ uniquely. This is because the process of taking the partial trace discards information about correlations (including quantum entanglement and classical correlations) between subsystems A and B. Many different joint states $\rho_{AB}$ can result in the same reduced state $\rho_A$. For example, both entangled and separable states can have the same marginal.

**What information is lost and what's needed:**  
 Additional information would be required to fully reconstruct $\rho_{AB}$ from $\rho_A$ alone. This could include, for example:
   - The explicit form of correlations with subsystem B,
   - Knowledge that the global state is pure, and possibly side information about how purification was chosen,
   - Complete tomography data on the global system,
   - Or knowing the full set of joint measurement statistics.
 
In summary, reconstructing $\rho_{AB}$ from just $\rho_A$ is not generally possible without more input, because the mapping $\rho_{AB} \to \rho_A$ via partial trace is many-to-one and irreversible.


In [155]:
def canonical_purification(rhoA: np.ndarray, tol=1e-12):
    """
    Given rhoA (nA x nA Hermitian, trace-1), return:
      - Psi_AB : a vector of length nA*nB representing the purification in A⊗B
                using the computational basis on B (nB = nA).
      - The eigenvalues, eigenvectors and rank
    We define tol to filter small eigenvalues.
    """
    # Spectral decomposition: rhoA = sum_k (lambdas[k]) |u_k><u_k|
    lambdas, U = np.linalg.eigh(rhoA)
    # Sort descending
    idx = np.argsort(lambdas)[::-1]
    lambdas = lambdas[idx]
    U = U[:, idx]
    # Keep only significant eigenvalues ( > tol )
    keep = lambdas > tol
    lambdas = lambdas[keep]
    U = U[:, keep]
    rank = len(lambdas)
    # Construct purified state in H_A ⊗ H_B (both dim nA)
    # |Psi> = sum_k sqrt(lambdas[k]) |u_k> ⊗ |k>_B
    nA = rhoA.shape[0]
    # Purified state as vector in nA*nB
    Psi_AB = np.zeros((nA * rank), dtype=complex)
    # Each term: sqrt(lambda_k) * u_k ⊗ |k>
    # |u_k> is nA x 1; |k>_B is computational basis in H_B
    Psi_AB = np.zeros((nA * rank), dtype=complex)
    for k in range(rank):
        v = U[:, k] # nA vector
        # sqrt(lambda_k) * v ⊗ |k>
        # position in AB for (i (A), k (B)): i*rank + k
        # We'll use np.kron to build the full vector
        basisB = np.zeros(rank)
        basisB[k] = 1.0
        Psi_AB += np.sqrt(lambdas[k]) * np.kron(v, basisB)
    results = {
        'Psi_AB': Psi_AB,
        'lambdas': lambdas,
        'U': U,
        'rank': rank
    }
    return results

In [158]:
# Example usage: assume rhoA already available from previous steps (2x2)
# If you only have rhoAB, compute rhoA = partial_trace_B(rhoAB) similarly
# Here we demonstrate with rhoA variable.

# Example: use the rhoA from earlier parts (replace with your rhoA)
# rhoA = <computed from question 3>

# You should have defined rhoA in previous question
# Let's define a test rhoA and compute its canonical purification
import numpy as np

# Example: pure state density matrix |0><0|
rhoA = np.array([[1, 0],
                 [0, 0]], dtype=complex)
# Or: mixed state (replace with your rhoA from earlier if desired)
# rhoA = np.array([[0.7, 0.3],
#                  [0.3, 0.3]], dtype=complex)

tol = 1e-12  # Small threshold for eigenvalues

purification_results = canonical_purification(rhoA, tol=tol)
Psi_AB = purification_results['Psi_AB']
lambdas = purification_results['lambdas']
U = purification_results['U']
rank = purification_results['rank']


# Verify by partial trace
# Reshape Psi_AB into (nA, nB) where nB = rank (since canonical_purification does this)
nA = rhoA.shape[0]
nB = rank
Psi_AB_matrix = Psi_AB.reshape((nA, nB))

# Compute the reduced density matrix on A by tracing out B
# rhoA_prime = Tr_B(|Psi_AB><Psi_AB|)
rhoA_prime = np.zeros((nA, nA), dtype=complex)
for i in range(nA):
    for j in range(nA):
        # sum over k (B)
        rhoA_prime[i, j] = np.dot(Psi_AB_matrix[i, :].conj(), Psi_AB_matrix[j, :])

# Compare with original rhoA
print("Original rhoA:\n", rhoA)
print("Reduced rhoA from purification:\n", rhoA_prime)
print("Are they close? ", np.allclose(rhoA, rhoA_prime, atol=1e-10))

# Show the purification vector in the computational basis
# Print the canonical purification vector |Psi_AB> in the computational basis
# The computational basis elements are |i>_A ⊗ |j>_B
print("Canonical purification |Psi_AB> in computational basis (flattened):")
print(Psi_AB)

# Optionally, display in ket notation for each non-zero component
print("\nNonzero components in |Psi_AB> = sum_{i,j} Psi_AB[i * nB + j] |i>_A |j>_B :")
for i in range(nA):
    for j in range(nB):
        coeff = Psi_AB[i * nB + j]
        if np.abs(coeff) > 1e-12:
            print(f"({coeff}) |{i}>_A |{j}>_B")


Original rhoA:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
Reduced rhoA from purification:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
Are they close?  True
Canonical purification |Psi_AB> in computational basis (flattened):
[1.+0.j 0.+0.j]

Nonzero components in |Psi_AB> = sum_{i,j} Psi_AB[i * nB + j] |i>_A |j>_B :
((1+0j)) |0>_A |0>_B


In [159]:
# After getting Psi_AB from canonical_purification, verify its properties to see if it is a valid purification

# Reshape Psi_AB to (nA, nB) for readability, where nA and nB can be obtained from rhoA and the rank:
nA = rhoA.shape[0]
nB = canonical_purification(rhoA)['rank']  # Or use the previously determined 'rank'
Psi_AB_matrix = Psi_AB.reshape((nA, nB))
print("Psi_AB (reshaped):\n", Psi_AB_matrix)

# Check properties

# 1. Psi_AB is normalized
norm = np.vdot(Psi_AB, Psi_AB)
print("Norm of |Psi_AB>:", norm)

# 2. The reduced density matrix on A equals rhoA (i.e., Tr_B(|Psi_AB><Psi_AB|) == rhoA)
rhoA_prime = np.zeros((nA, nA), dtype=complex)
for i in range(nA):
    for j in range(nA):
        # sum over B
        rhoA_prime[i, j] = np.dot(Psi_AB_matrix[i, :].conj(), Psi_AB_matrix[j, :])
print("Original rhoA:\n", rhoA)
print("Reduced rhoA from purification:\n", rhoA_prime)
print("Purification check (close to original)?", np.allclose(rhoA, rhoA_prime, atol=1e-10))

Psi_AB (reshaped):
 [[1.+0.j]
 [0.+0.j]]
Norm of |Psi_AB>: (1+0j)
Original rhoA:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
Reduced rhoA from purification:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
Purification check (close to original)? True
